In [1]:
import re

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.metrics import pairwise_distances

In [2]:
def get_weighted_costs(file_paths):
    costs = []
    for index, cost_path in enumerate(file_paths):
        dataset, algorithm, k = re.findall(r"/.+/(.+)/(.+)-k(\d+)-", str(cost_path))[0]
        with open(cost_path, "r") as f:
            weighted_cost = float(f.read())
        costs.append({
            "dataset": dataset,
            "algorithm": algorithm,
            "k": k,
            "cost": weighted_cost,
            "file_path": str(cost_path)
        })
    return pd.DataFrame(costs)

In [3]:
data_results_dir = Path("../data/results/experiments/enron/")

In [4]:
cost_paths = list(data_results_dir.glob("**/real_cost.txt"))

In [5]:
len(cost_paths)

120

In [6]:
df_cost_data = get_weighted_costs(cost_paths)

In [7]:
df_aggr_cost_data = df_cost_data.groupby(["dataset", "algorithm", "k"], as_index=False).agg(
    cost_count=("cost", "count"),
    cost_mean=("cost", "mean"),
    cost_std=("cost", "std")
)
df_aggr_cost_data

,dataset,algorithm,k,cost_count,cost_mean,cost_std
0,enron,bico,10,20,2.352009e+07,290024.801605
1,enron,bico,20,20,2.176014e+07,237554.826471
2,enron,bico,30,20,2.046459e+07,253002.770144
3,enron,bico,40,20,1.948088e+07,180602.641830
4,enron,bico,50,19,1.869068e+07,256030.698589
5,enron,group-sampling,10,2,2.341216e+07,102689.753803
6,enron,sensitivity-sampling,10,4,2.324840e+07,73269.590511
7,enron,sensitivity-sampling,20,4,2.170139e+07,211228.191283
8,enron,sensitivity-sampling,30,4,2.015484e+07,332529.492967
9,enron,sensitivity-sampling,40,3,1.941674e+07,65053.264795


In [8]:
df = pd.pivot_table(df_aggr_cost_data, values="cost_mean", index=["algorithm"], columns=["k"])
df#.style.format(precision=0, thousands=",").highlight_min(color = 'lightgreen', axis = 0)

k,10,20,30,40,50
algorithm,,,,,
bico,2.352009e+07,2.176014e+07,2.046459e+07,1.948088e+07,1.869068e+07
group-sampling,2.341216e+07,NaN,NaN,NaN,NaN
sensitivity-sampling,2.324840e+07,2.170139e+07,2.015484e+07,1.941674e+07,1.873558e+07


In [9]:
#df = pd.pivot_table(df_aggr_cost_data, values="cost_std", index=["algorithm"], columns=["k"])
#df.style.format(precision=0, thousands=",").highlight_min(color = 'lightgreen', axis = 0)